# Simple RAG System for SpaceX Projects
## Part (1) Data Acquisition and Preprocessing   
\

## Objective
### Develop a basic Retrieval-Augmented Generation (RAG) system using a diverse dataset to answer questions specifically related to SpaceX.

The primary goal of this project is to build a RAG system that can respond to user queries about SpaceX, their ongoing projects, and their latest space missions by utilizing a combination of different data sources.

\
## Dataset Acquisition and Processing
In this project, various datasets are collected from multiple resources, processed, and stored in my Google Drive for easy access and use. The collected data focuses on providing comprehensive information relevant to SpaceX and its activities.

\
### Data Sources
The dataset for this RAG system is derived from four main types of sources, each contributing unique information to form a complete and informative resource:

- **PDF Files**: These include SpaceX mission guides, general information about SpaceX's rocket fleet, and detailed documentation about the company's history, vision, and mission. The PDFs are preprocessed, split into smaller chunks, and stored as text files.
  
- **Wikipedia Articles**: Articles sourced from Wikipedia provide scientific insights and background on topics related to space exploration, space vehicles, rocket technology, and historical space missions.

- **CC News Articles**: This data source comprises journalistic articles covering SpaceX, NASA, and other prominent space organizations. These articles highlight key achievements, collaborations, and the impact of SpaceX on the space industry.

- **Video Transcripts**: Videos related to space exploration, SpaceX’s ambitious goals, and the intricacies of rocket engines are transcribed. The transcripts offer a detailed, comprehensive view of the company’s work and its future vision for space travel and technology.

\
## Preprocessing and Storage
After collecting data from the above sources, each resource was preprocessed using text splitting, cleaning, and formatting techniques. The processed data was then stored in `.txt` files for seamless retrieval and integration into the RAG system.

\
## Purpose and Use Case
The resulting RAG system is designed to serve as a Q&A service, focusing on providing accurate and contextual information regarding SpaceX's endeavors, current projects, and their broader implications for space exploration. By leveraging this system, users can gain deeper insights into the company's efforts and keep up-to-date with their latest achievements and missions.

In [16]:
# install the required libraries to download datasets from
!pip install datasets gdown ffmpeg-python


In [17]:
# install wisper model will be needed in video translation
!pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-u8ugmqg8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-u8ugmqg8
  Resolved https://github.com/openai/whisper.git to commit 423492dda7806206abe56bdfe427c1096473a020
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240927-py3-none-any.whl size=802818 sha256=1ccaa3538050b825a465b898b9defeebb9db986ae51386f1549af3fb0bf4245a
  Stored in directory: /tmp/pip-ephem-wheel-cache-4vsi5i3r/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [21]:
# import the required functions

from google.colab import drive
from datasets import load_dataset
import whisper
import gdown
import ffmpeg
import os
import re


In [4]:
# mount google drive to load and upload the data
drive.mount('/content/drive')

# change the divercory of working to RagProject folder to make it easier to work with the dirve
os.chdir("/content/drive/MyDrive/RagProject ")

Mounted at /content/drive


## Datasets from wikipedia and cc_new process and download

In [5]:
# the following key words should be mensioned in the title of the dataset to be selected as relevant topic

space_keywords = [
    "SpaceX", "Raptor engine", "Falcon 9", "Falcon Heavy", "Elon Musk",
    "rocket engine", "aerospace", "space exploration", "Mars mission",
    "reusable rocket", "rocket technology", "space mission",
    "Raptor rocket engine", "rocket propulsion", "Raptor vacuum",
    "spacecraft", "launch vehicle", "space launch", "orbital rocket",
    "Falcon rocket", "Merlin engine", "Starship"
]


In [6]:
# import cc_news_dataset with 30% of the total data for the porposes of computational time
cc_news_dataset = load_dataset("cc_news", split="train[:30%]")

# import wikipedia_dataset with 30% of the total data for the porposes of computational time
# this dataset were released on 1st of march 2022 in English language
wiki_dataset = load_dataset("wikipedia", "20220301.en", split="train[:30%]")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.57k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/211M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/234M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/219M [00:00<?, ?B/s]

...

train-00038-of-00041.parquet:   0%|          | 0.00/203M [00:00<?, ?B/s]

train-00039-of-00041.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

train-00040-of-00041.parquet:   0%|          | 0.00/185M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6458670 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
# navigate through the dataset
wiki_dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1937601
})

In [8]:
# navigate through the dataset
wiki_dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1937601
})

## Process and filter the required articles from the datasets

In [9]:

# create a regex pattern that matches any of the keywords as whole words
pattern = r'\b(' + '|'.join(re.escape(keyword) for keyword in space_keywords) + r')\b'

# create the filtersted wiki_articles and its titles
filtered_wiki_titles = []
filtered_wiki_articles = []

for i, j in enumerate(wiki_dataset):

  if re.search(pattern, j["title"], re.IGNORECASE):
    filtered_wiki_titles.append(j["title"])
    filtered_wiki_articles.append(j["text"])

# create the filtersted cc_news and its titles
filtered_cc_news_articles=[]
filtered_cc_news_titles=[]

for i, j in enumerate(cc_news_dataset):
  if re.search(pattern, j["title"], re.IGNORECASE):
    filtered_cc_news_titles.append(j["title"])
    filtered_cc_news_articles.append(j["text"])



In [10]:
def clean_text(text):

    # replace multiple periods ('..') with a single space
    text = re.sub(r'\.{2,}', ' ', text)
    # replace newline characters with a space
    text = text.replace('\n', ' ')
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [12]:
# save filtered CC News titles and articles to separate text files

with open('DownloadedDataSet/filtered_cc_news_titles.txt', 'w', encoding='utf-8') as file:
    for title in filtered_cc_news_titles:
        file.write(clean_text(title) + '\n\n\n*')
with open('textDataSet/filtered_cc_news_articles.txt', 'w', encoding='utf-8') as file:
    for article in filtered_cc_news_articles:
        file.write(clean_text(article) + '\n\n\n*')



# save filtered wiki articles and titles to separate text files

with open('DownloadedDataSet/filtered_wiki_titles.txt', 'w', encoding='utf-8') as file:
    for title in filtered_wiki_titles:
        file.write(clean_text(title) + '\n\n\n*')
with open('textDataSet/filtered_wiki_articles.txt', 'w', encoding='utf-8') as file:
    for article in filtered_wiki_articles:
        file.write(clean_text(article) + '\n\n\n*')



## Download the processed (wikipedia and cc_news) files

In [15]:

# downloading the previously processed files
gdown.download('https://drive.google.com/uc?id=1WvGZ01SNch4uLYympd3gB1Qm2lW-IhWj', 'wiki_articles.txt', quiet=False)
gdown.download('https://drive.google.com/uc?id=1CoBdWYujQMXVt-C8DRcq6cujKytyPGVh', 'cc_news_articles.txt', quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1WvGZ01SNch4uLYympd3gB1Qm2lW-IhWj
To: /content/drive/MyDrive/RagProject /wiki_articles.txt
100%|██████████| 537k/537k [00:00<00:00, 49.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CoBdWYujQMXVt-C8DRcq6cujKytyPGVh
To: /content/drive/MyDrive/RagProject /cc_news_articles.txt
100%|██████████| 2.95M/2.95M [00:00<00:00, 112MB/s]


'cc_news_articles.txt'

## Process the Video files

In [20]:
# this function is used to extract audio from the video

def extract_audio(video_file, output_audio_file):
    (
        ffmpeg
        .input(video_file)
        .output(output_audio_file, format='mp3')
        .run(overwrite_output=True)
    )


In [22]:

# load the Whisper model and chose the base model
model = whisper.load_model("base")

# transcribe the audio file to txt
def transcribe_audio(audio_file):
    result = model.transcribe(audio_file)
    return result['text']


100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 41.3MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [23]:

# this function is used to convert the video folder into txt files in the provided pathes

def process_video_to_text(video_dir, output_dir):
    for i in os.listdir(video_dir):
        if i.endswith(".mp4"):
            # Construct the correct video and output audio paths for each file
            video_path = os.path.join(video_dir, i)
            audio_path = os.path.join(output_dir, i.split(".")[0] + ".mp3")

            print("start_processing_video:", i)

            # Extract audio
            extract_audio(video_path, audio_path)

            # Transcribe the audio
            transcript = transcribe_audio(audio_path)

            # Save the transcription to a text file
            with open(os.path.join(output_dir, i.split(".")[0] + ".txt"), "w") as file:
                file.write(transcript)

            print("finished_video:", i)


In [24]:

# convert the videos to txt transcript
process_video_to_text("VideosDataSet/","soundDataSet/")


start_processing_video: The Real Reason SpaceX Developed The Raptor Engine!.mp4


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


finished_video: The Real Reason SpaceX Developed The Raptor Engine!.mp4
start_processing_video: The Journey of Elon Musk (Documentary).mp4


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


finished_video: The Journey of Elon Musk (Documentary).mp4
start_processing_video: Raptor Engine.mp4


...

finished_video: SpaceX Raptor3.mp4
start_processing_video: SpaceX moonBase.mp4


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


finished_video: SpaceX moonBase.mp4
start_processing_video: How SpaceX and NASA Plan To Build A Mars Colony!.mp4


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


finished_video: How SpaceX and NASA Plan To Build A Mars Colony!.mp4
